In [1]:
import os
import time
from openai import OpenAI
import datetime

In [6]:
os.environ["OPENAI_API_KEY"] = ''
# Set MODEL to "gpt-3.5-turbo"
model = "gpt-3.5-turbo"
# Set temperature to 0.9, it can be from 0~1, 0 is the most conservative, 1 is the most creative
temperature = 0.9
output_file = 'results.txt'

client = OpenAI(
    # This is the default and can be omitted
    api_key=os.environ.get("OPENAI_API_KEY"),
)

In [16]:
def grab_fileinfo(dict, key, data_folder):
    data = ''
    
    # Current working directory
    current_dir = os.getcwd()
    path = os.path.join(current_dir, data_folder, dict[key])

    with open(path, 'r') as file:
        data = file.read()
    return data

def create_message(role, input):
    message_list = []
    system_message = {"role": "system", "content": role}
    message_list.append(system_message)
    user_message = {"role": "user", "content": input}
    message_list.append(user_message)
    return message_list

def grab_respons(message_list):
    chat_completion = client.chat.completions.create(
    messages=message_list,
    model="gpt-3.5-turbo",
    )
    return chat_completion.choices[0].message.content

def create_cover_msg(messagingDict, resume_key):
    msgstr = append_job(messagingDict, resume_key)
    msgstr += 'Cover Letter:\n'
    msgstr += messagingDict['CoverLetter']
    return msgstr

def append_job(messagingDict, resume_key):
    msgstr = 'Resume:\n'
    msgstr += messagingDict[resume_key]
    msgstr += 'Job Listing:\n'
    msgstr += messagingDict['JobListing']
    return msgstr

def full_sweep(messagingDict):
    message_list = create_message(messagingDict['CreatePrompt'], messagingDict['ResumeInfo'])
    messagingDict['ResumeInfo2'] = grab_respons(message_list)
    msg = append_job(messagingDict, 'ResumeInfo2')
    message_list = create_message(messagingDict['RefinePrompt'], msg)
    messagingDict['ResumeInfo3'] = grab_respons(message_list)
    msg = append_job(messagingDict, 'ResumeInfo3')
    message_list = create_message(messagingDict['AuditPrompt'], msg)
    output = {}
    output['FinalResume'] = messagingDict['ResumeInfo3']
    output['Rating'] = grab_respons(message_list)
    msg = create_cover_msg(messagingDict, 'ResumeInfo3')
    message_list = create_message(messagingDict['CoverPrompt'], msg)
    output['CoverLetter'] = grab_respons(message_list)
    
    return output

In [8]:
messagingDict = {}
data_folder = 'data'

filenames = {
    'CreatePrompt' : 'create_prompt.txt',
    'RefinePrompt' : 'refine_prompt.txt',
    'AuditPrompt' : 'audit_prompt.txt',
    'ResumeInfo' : 'resume_info.txt',
    'CoverLetter' : 'cover_letter.txt',
    'CoverPrompt' : 'cover_prompt.txt',
    'JobListing' : 'job_listing.txt'
}

for k in filenames.keys():
    messagingDict[k] = grab_fileinfo(filenames, k, data_folder)

messagingDict['Separator'] = '\n-----------------------------------------------------------------------\n'

In [18]:
results = full_sweep(messagingDict)

In [22]:
try:
    #datetime_object = str(datetime.datetime.fromtimestamp(time.time()))
    f = open("resume_file.txt", "w")
    f.write(results['FinalResume'])
    f.close()
    f = open("rating_file.txt", "a")
    f.write(results['Rating'])
    f.close()
    f = open("cover_output.txt", "w")
    f.write(results['CoverLetter'])
    f.close()
except IOError:
    print("Could not write file")
finally:
    f.close()